## O Primeiro Notebook (Extração e Limpeza)

## 0 - Diagnóstico
#### Analisar os nomes das colunas do arquivo.

In [1]:
import pandas as pd

# Caminho do arquivo (ajuste se necessário)
path = '../data/raw/ebt_municipios_br.xlsx'

# Carrega sem cabeçalho para vermos as primeiras linhas brutas
df_debug = pd.read_excel(path, header=None)

print("--- 5 PRIMEIRAS LINHAS DO ARQUIVO ---")
display(df_debug.head(5))

print("\n--- NOMES DAS COLUNAS ATUAIS ---")
# Carrega normal para ver o que ele "achou" que eram as colunas
df_normal = pd.read_excel(path)
print(list(df_normal.columns))

--- 5 PRIMEIRAS LINHAS DO ARQUIVO ---


,0,1,2,3,4,5,6,7,8,9,...,108,109,110,111,112,113,114,115,116,117
0,Ranking,Nota Avaliação,UF,Entidade Avaliada,Código IBGE,População,1 - LOCALIZAÇÃO DO SÍTIO OFICIAL: O ente feder...,Nota Questão,1.1 - Informe o link do sítio oficial:,Nota Questão,...,25.2 - O pedido 3 foi respondido no prazo?,Nota Questão,"25.3 - O ente federado atendeu ao pedido, resp...",Nota Questão,"25.4 - Na resposta ao pedido 3, o ente federad...",Nota Questão,26 - É possível realizar o acompanhamento elet...,Nota Questão,26.1 - Indique o(s) recurso(s) disponibilizado...,Nota Questão
1,25,6.53,Acre,Acre,NaN,0,Sim,"0,00",http://acre.gov.br/,0,...,Não,0,Não,0,Não,0,Sim,"0,40",Data de registro da solicitação; Situação da s...,0
2,8,9.75,Alagoas,Alagoas,NaN,0,Sim,"0,00",www.agenciaalagoas.al.gov.br,0,...,Sim,0,Sim,0,Sim,0,Sim,"0,40",Data de registro da solicitação; Situação da s...,0
3,20,8.67,Amazonas,Amazonas,NaN,0,Sim,"0,00",http://www.amazonas.am.gov.br/,0,...,Sim,0,Sim,0,Não,0,Sim,"0,40",Data de registro da solicitação; Situação da s...,0
4,6,9.83,Amapá,Amapá,NaN,0,Sim,"0,00",https://www.portal.ap.gov.br/,0,...,Sim,0,Sim,0,Sim,0,Sim,"0,40",Data de registro da solicitação; Situação da s...,0



--- NOMES DAS COLUNAS ATUAIS ---
['Ranking', 'Nota Avaliação', 'UF', 'Entidade Avaliada', 'Código IBGE', 'População', '1 - LOCALIZAÇÃO DO SÍTIO OFICIAL: O ente federado possui sítio oficial localizado na internet? ', 'Nota Questão', '1.1 - Informe o link do sítio oficial:', 'Nota Questão.1', '2 - LOCALIZAÇÃO DE PORTAL DE TRANSPARÊNCIA: O ente federado divulga informações orçamentárias e financeiras consolidadas em um portal de transparência?', 'Nota Questão.2', '2.1 - Informe o link do portal de transparência:', 'Nota Questão.3', '3 - ESTRUTURA ORGANIZACIONAL: O ente federado disponibiliza informações sobre asua estrutura organizacional?', 'Nota Questão.4', '4 - UNIDADES ADMINISTRATIVAS: O ente federado disponibiliza informações sobre as suas unidades administrativas?', 'Nota Questão.5', '4.1 - Indique as informações sobre as unidades administrativas que o ente federado disponibiliza:', 'Nota Questão.6', '5 - RECEITAS: O ente federado disponibiliza informações sobre Receitas?', 'Nota 

In [3]:
import pandas as pd
import os

# ==========================================
# 1. CONFIGURAÇÃO
# ==========================================
RAW_DATA_PATH = '../data/raw/ebt_municipios_br.xlsx'
PROCESSED_DATA_PATH = '../data/processed/rj_ebt_limpo.csv'

# ==========================================
# 2. CARGA (ETL: EXTRACT)
# ==========================================
print("🔄 Carregando dados...")
if not os.path.exists(RAW_DATA_PATH):
    raise FileNotFoundError(f"Arquivo não encontrado: {RAW_DATA_PATH}")

# Lê o Excel
df_br = pd.read_excel(RAW_DATA_PATH)

# ==========================================
# 3. TRANSFORMAÇÃO (ETL: TRANSFORM)
# ==========================================
# 3.1 Limpeza dos Nomes das Colunas (Remove espaços extras nas pontas)
df_br.columns = [c.strip() for c in df_br.columns]

# 3.2 Mapeamento Explícito (Baseado na sua lista)
# Vamos renomear para facilitar o uso no código (Snake Case)
rename_map = {
    'UF': 'uf',
    'Entidade Avaliada': 'municipio',
    'Nota Avaliação': 'nota',
    'Ranking': 'ranking_nacional'
}
df_br = df_br.rename(columns=rename_map)

print(f"📊 Colunas renomeadas: {list(df_br.columns)[:5]}...")

# 3.3 FILTRO DO ESTADO (A CORREÇÃO PRINCIPAL) 🚨
# O dataset usa "Rio de Janeiro" e não "RJ".
# Usamos .strip() no valor também para garantir que não haja espaços "Rio de Janeiro "
df_rj = df_br[df_br['uf'].str.strip() == 'Rio de Janeiro'].copy()

print(f"📍 Total de registros encontrados no Rio de Janeiro: {df_rj.shape[0]}")

# 3.4 Identificar Paracambi
# Convertemos para string e forçamos maiúscula/minúscula para garantir o 'match'
df_rj['is_paracambi'] = df_rj['municipio'].apply(lambda x: 'Sim' if 'Paracambi' in str(x) else 'Não')

# 3.5 Recalcular Ranking Estadual
# Ordena da maior nota para a menor
df_rj_sorted = df_rj.sort_values(by='nota', ascending=False).reset_index(drop=True)
df_rj_sorted['ranking_estadual'] = df_rj_sorted.index + 1

# 3.6 Extrair Estatísticas de Paracambi
paracambi_stats = df_rj_sorted[df_rj_sorted['is_paracambi'] == 'Sim']

# ==========================================
# 4. CARGA (ETL: LOAD)
# ==========================================
df_rj_sorted.to_csv(PROCESSED_DATA_PATH, index=False)
print(f"💾 Dados limpos salvos em: {PROCESSED_DATA_PATH}")

# ==========================================
# 5. ANÁLISE INICIAL
# ==========================================
print("\n--- 🔍 RESULTADO PARACAMBI ---")
if not paracambi_stats.empty:
    display(paracambi_stats[['ranking_estadual', 'municipio', 'nota', 'ranking_nacional']])
else:
    print("⚠️ Ainda não encontrei Paracambi. Verifique abaixo os nomes disponíveis no RJ:")
    print(df_rj_sorted['municipio'].unique())

print("\n--- 🏆 TOP 5 ESTADUAL (O ALVO) ---")
display(df_rj_sorted.head(5)[['ranking_estadual', 'municipio', 'nota']])

🔄 Carregando dados...
📊 Colunas renomeadas: ['ranking_nacional', 'nota', 'uf', 'municipio', 'Código IBGE']...
📍 Total de registros encontrados no Rio de Janeiro: 39
💾 Dados limpos salvos em: ../data/processed/rj_ebt_limpo.csv

--- 🔍 RESULTADO PARACAMBI ---


,ranking_estadual,municipio,nota,ranking_nacional
21,22,Paracambi - RJ,6.11,385



--- 🏆 TOP 5 ESTADUAL (O ALVO) ---


,ranking_estadual,municipio,nota
0,1,São Pedro da Aldeia - RJ,10.00
1,2,Niterói - RJ,10.00
2,3,Mesquita - RJ,10.00
3,4,Japeri - RJ,9.21
4,5,Duque de Caxias - RJ,9.21
